# Содержание
1. [Обзор данных](#1)
2. [Предобработка и исследовательский анализ данных](#2) 
3. [Формулировка ML-задачи на основе бизнес-задачи](#3)
4. [Разработка модели ML](#4)
5. [Портрет «ненадёжного» клиента](#5)
6. [Общий вывод](#6)

# Прогнозирование оттока клиентов в сети отелей «Как в гостях»

**Заказчик исследования** — сеть отелей «Как в гостях». 

Чтобы привлечь клиентов, эта сеть отелей добавила на свой сайт возможность забронировать номер без предоплаты. Однако если клиент отменял бронирование, то компания терпела убытки. Сотрудники отеля могли, например, закупить продукты к приезду гостя или просто не успеть найти другого клиента.

**Цель исследования** - разработать систему, которая предсказывает отказ от брони.

Если модель покажет, что бронь будет отменена, клиенту предлагается внести депозит. Размер депозита — 80% от стоимости номера за одни сутки и затрат на разовую уборку. Деньги будут списаны со счёта клиента, если он всё же отменит бронь.

**Задачи:**

- подготовить данные;
- построить две модели линейной регрессии на разных наборах данных:
  - используя все данные из файла,
  - используя только числовые переменные, исключив категориальные;
- сравнить результаты работы линейной регрессии на двух наборах данных по метрикам RMSE, MAE и R2.

**Описание данных**

В таблицах hotel_train и hotel_test содержатся одинаковые столбцы:


- id — номер записи;
- adults — количество взрослых постояльцев;
- arrival_date_year — год заезда;
- arrival_date_month — месяц заезда;
- arrival_date_week_number — неделя заезда;
- arrival_date_day_of_month — день заезда;
- babies — количество младенцев;
- booking_changes — количество изменений параметров заказа;
- children — количество детей от 3 до 14 лет;
- country — гражданство постояльца;
- customer_type — тип заказчика:
    - Contract — договор с юридическим лицом;
    - Group — групповой заезд;
    - Transient — не связано с договором или групповым заездом;
    - Transient-party — не связано с договором или групповым заездом, но связано с бронированием типа Transient.
- days_in_waiting_list — сколько дней заказ ожидал подтверждения;
- distribution_channel — канал дистрибуции заказа;
- is_canceled — отмена заказа;
- is_repeated_guest — признак того, что гость бронирует номер второй раз;
- lead_time — количество дней между датой бронирования и датой прибытия;
- meal — опции заказа:
    - SC — нет дополнительных опций;
    - BB — включён завтрак;
    - HB — включён завтрак и обед;
    - FB — включён завтрак, обед и ужин.
- previous_bookings_not_canceled — количество подтверждённых заказов у клиента;
- previous_cancellations — количество отменённых заказов у клиента;
- required_car_parking_spaces — необходимость места для автомобиля;
- reserved_room_type — тип забронированной комнаты;
- stays_in_weekend_nights — количество ночей в выходные дни;
- stays_in_week_nights — количество ночей в будние дни;
- total_nights — общее количество ночей;
- total_of_special_requests — количество специальных отметок.

## Обзор Данных <a id="1"></a>

### Подключение необходимых библиотек.

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from sklearn.preprocessing import OrdinalEncoder 
from sklearn.preprocessing import OneHotEncoder

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler


from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve

In [2]:
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

### Открытие и изучение данных.

In [3]:
server_path = '/datasets/'
local_path = ''
train = 'hotel_train.csv'
test = 'hotel_test.csv'

try:
    data_train = pd.read_csv(server_path + train)  
    data_test = pd.read_csv(server_path + test)
except: 
    data_train = pd.read_csv(local_path + train)  
    data_test = pd.read_csv(local_path + test)

In [4]:
data_train.head(5)

,id,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,booking_changes,days_in_waiting_list,customer_type,required_car_parking_spaces,total_of_special_requests,total_nights
0,0,0,7.0,2015,July,27,1,0,1,1.0,0.0,0.0,BB,GBR,Direct,0,0,0,A,0,0,Transient,0,0,1
1,1,0,14.0,2015,July,27,1,0,2,2.0,0.0,0.0,BB,GBR,TA/TO,0,0,0,A,0,0,Transient,0,1,2
2,2,0,0.0,2015,July,27,1,0,2,2.0,0.0,0.0,BB,PRT,Direct,0,0,0,C,0,0,Transient,0,0,2
3,3,0,9.0,2015,July,27,1,0,2,2.0,0.0,0.0,FB,PRT,Direct,0,0,0,C,0,0,Transient,0,1,2
4,4,1,85.0,2015,July,27,1,0,3,2.0,0.0,0.0,BB,PRT,TA/TO,0,0,0,A,0,0,Transient,0,1,3


In [5]:
data_test.head(5)

,id,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,booking_changes,days_in_waiting_list,customer_type,required_car_parking_spaces,total_of_special_requests,total_nights
0,6086,1,74.0,2017,January,1,1,1,0,2.0,0.0,0.0,BB,PRT,TA/TO,0,0,0,A,0,0,Transient,0,0,1
1,6087,1,62.0,2017,January,1,1,2,2,2.0,0.0,0.0,BB,AUT,TA/TO,0,0,0,A,0,0,Transient,0,1,4
2,6088,1,62.0,2017,January,1,1,2,2,2.0,0.0,0.0,BB,AUT,TA/TO,0,0,0,A,0,0,Transient,0,1,4
3,6089,1,71.0,2017,January,1,1,2,2,1.0,0.0,0.0,BB,PRT,TA/TO,0,0,0,A,0,0,Transient,0,1,4
4,6090,1,172.0,2017,January,1,1,2,5,2.0,0.0,0.0,BB,BEL,TA/TO,0,0,0,A,0,0,Transient,0,0,7


In [6]:
#data_train.info()

In [7]:
#data_test.info()

Пропусков не выявлено. Проверим данные на наличие дубликатов.

In [8]:
def unique_check(data):
    columns = data.columns
    for c in columns:
        print(c.upper(), '\n', data[c].unique(), '\n')

In [9]:
#unique_check(data_train)

In [10]:
#unique_check(data_test)

In [11]:
data_train.duplicated().sum()

0

In [12]:
data_test.duplicated().sum()

0

Дубликатов не обнаружено.

Целевой признак для модели - столбец is_canceled. Проверим баланс классов для обучения.

In [13]:
data_train['is_canceled'].value_counts()

0    41185
1    24044
Name: is_canceled, dtype: int64

## Вывод по шагу 1.

В некоторых столбцах (lead_time, adults, children, babies) следует заменить тип данных с float на int для удобства дальнейшего исследования. В значениях столбцов reserved_room_type и meal нужно убрать пробелы. Пропусков и дубликатов в данных нет. Перед обучением модели классы следует сбалансировать. В следующем шаге проведем исследовательский анализ.

## Предобработка и исследовательский анализ данных <a id="2"></a>

Заменим типы данных.

In [14]:
columns = ['lead_time', 'adults', 'children', 'babies']
for c in columns:
    data_train[c] = data_train[c].astype('int64')
    data_test[c] = data_test[c].astype('int64')

Исправим значения столбца 'reserved_room_type' и 'meal'.

In [15]:
data_train['reserved_room_type'] = data_train['reserved_room_type'].str.strip()
data_train['meal'] = data_train['meal'].str.strip()
data_test['reserved_room_type'] = data_test['reserved_room_type'].str.strip()
data_test['meal'] = data_test['meal'].str.strip()

Произведем на этом этапе прямое кодирование, чтобы захватить больше данных при анализе.

In [16]:
data_ohe = pd.get_dummies(data_train, drop_first=True)

In [17]:
#data_ohe.head(5)

In [18]:
#data_ohe.describe().T

- В среднем клиенты бронируют номера за три месяца до заезда.
- В данных представлены данные за два года - 2015 и 2016, распределены равномерно по месяцам и дням месяца.
- Гости бронируют гостиницу на срок от одного до 10 дней, в большинстве случаев - на три дня.
- Номера бронируются в основном на двух взрослых, есть необычное значение на номера для гостей с 10-ю младенцами.
- В основном бронь происходит от новых гостей, но есть и те, кто постоянно возвращается (58 раз).
- 1% гостей уже отменяли бронирование раньше, кто-то из гостей отменял бронирование уже 26 раз.
- В основном заказ подтверждается в течение трех дней.
- Бронирование парковки не пользуется популярностью среди гостей.

Странно, что в некоторых записях о бронировании количество взрослых равно нулю. Проверим, ошибка ли это.

In [19]:
#data_train.query('adults == 0')['children']

In [20]:
#data_train.query('adults == 0 & children == 0')['babies']

In [21]:
#data_train.query('adults == 0 & children == 0')

Похоже на аномалию. Заменим нули кол-вом взрослых = 2 (медианное по выборке).

In [22]:
data_train['adults'] = data_train['adults'].replace(0, np.nan).bfill()
data_train['adults'] = data_train['adults'].fillna(2)
data_test['adults'] = data_test['adults'].replace(0, np.nan).bfill()
data_test['adults'] = data_test['adults'].fillna(2)

In [23]:
data_test['adults'].value_counts()

2.0    24290
1.0     6289
3.0     1824
4.0        9
Name: adults, dtype: int64

Посмотрим на средние значения признаков в двух группах - тех, кто отменил бронирование и тех, кто заселился.

In [24]:
#data_train[data_train['is_canceled']==1].describe().T

In [25]:
#data_train[data_train['is_canceled']==0].describe().T

- По таблице видно, что чем больше дней остается до заезда, тем чаще бронирование отменяют.
- Логично, что часто отменяющие бронирование гости, отменили его и в этот раз.
- Среди отмененных записей клиент ожидал подверждения в среднем 5 дней, среди тех, кто бронь не отменил - 2 дня.

Построим столбчатые гистограммы и распределения признаков для тех, кто ушел и тех, кто остался.

In [26]:
#numeric = [column for column in data_train][2:4] + [column for column in data_train][5:12] + [column for column in data_train][15:18] + [column for column in data_train][19:21] + [column for column in data_train][22:]

In [27]:
#canceled = data_train[data_train['is_canceled']==1]
#stayed = data_train[data_train['is_canceled']==0]
#distplot_columns = numeric
#for column in distplot_columns: 
#  plt.figure(figsize=(10,4)) 
#  plt.title(column)
#  sns.distplot(canceled[column])
#  sns.distplot(stayed[column])
#  plt.legend(['Отток', 'Оставшиеся'])
#  plt.show()

Построим матрицу корреляции признаков.

In [28]:
#plt.figure(figsize=(20,20))
#sns.heatmap(data = data_train.corr(), annot=True, square=True, cmap='YlGnBu')
#plt.title('Матрица корреляций')
#plt.show()

Явной зависимости факта оттока от признаков не обнаружено.
Имеются два мультиколлинеарных признака - stays_in_week_nights и total_nights.

Cудя по описанию, total_nights — это сумма stays_in_week_nights и stays_in_weekend_nights, проверим так ли это.

In [29]:
l = set(list(data_train[['stays_in_week_nights','stays_in_weekend_nights']].sum(axis=1)))
ll = set(list(data_train['total_nights']))
if l == ll: 
    print("Lists are equal") 
else: 
    print("Lists are not equal")


Lists are equal


Удалим лишние столбцы и стоблбец id, он не несет значимости для модели.

In [30]:
data_train = data_train.drop(['stays_in_week_nights', 'stays_in_weekend_nights', 'id'], axis=1)
data_test = data_test.drop(['stays_in_week_nights', 'stays_in_weekend_nights', 'id'], axis=1)
data_train = data_train.drop_duplicates()
data_test = data_test.drop_duplicates()
print(data_train.duplicated().sum())
print(data_test.duplicated().sum())

0
0


### Вывод по шагу 3.

Данные готовы к решению задачи. Были удалены лишние столбцы, заменены типы данных, удалены пробелы в столбцах.

Был произведен исследовательский анализ данных, отмечены общие особенности по всей выборке, произведено сравнение средних значений признаков в двух группах - тех, кто отменил бронирование и тех, кто заселился, проверена корреляция признаков.
В данных имеются выбросы, но они не сильно влияют на среднее значение по выборке.

## Формулировка ML-задачи на основе бизнес-задачи <a id="3"></a>

В отеле есть несколько типов номеров. В зависимости от типа номера назначается стоимость за одну ночь. Есть также затраты на уборку. Если клиент снял номер надолго, то убираются каждые два дня. 

Стоимость номеров отеля:

- категория A: за ночь — 1 000, разовое обслуживание — 400;
- категория B: за ночь — 800, разовое обслуживание — 350;
- категория C: за ночь — 600, разовое обслуживание — 350;
- категория D: за ночь — 550, разовое обслуживание — 150;
- категория E: за ночь — 500, разовое обслуживание — 150;
- категория F: за ночь — 450, разовое обслуживание — 150;
- категория G: за ночь — 350, разовое обслуживание — 150.

В ценовой политике отеля используются сезонные коэффициенты: весной и осенью цены повышаются на 20%, летом — на 40%.
Убытки отеля в случае отмены брони номера — это стоимость одной уборки и одной ночи с учётом сезонного коэффициента.

Создадим функцию, присваивающую указанные цены по типу номера и функцию подсчета прибыли.

In [31]:
def prices(df):
    df['room_cost'] = df['reserved_room_type'].map({'A':1000,'B':800, 'C':600,'D':550,'E':500,'F':450,'G':350})
    df['cleaning_cost'] = df['reserved_room_type'].map({'A':400,'B':350, 'C':350,'D':150,'E':150,'F':150,'G':150})
    return df

In [32]:
def profit(row, deposite):
    
    room_cost = row['room_cost']
    cleaning_cost = row['cleaning_cost']
    winter = ['December', 'January', 'February']
    summer = ['June', 'July', 'August']
    
    n = row['total_nights']
    
    
    if row['is_canceled'] == 0:
        expenses = cleaning_cost*(1 + n // 2)
        
        if row['arrival_date_month'] in winter:
            revenue = room_cost*n 
                
        elif row['arrival_date_month'] in summer:   
            revenue = room_cost*n + room_cost*n*0.4
        
        else: 
            revenue = room_cost*n + room_cost*n*0.2
                
    else:
        
        if row['arrival_date_month'] in winter:
            expenses = cleaning_cost + room_cost
                
        elif row['arrival_date_month'] in summer:   
            expenses = cleaning_cost + room_cost + room_cost*0.4
        
        else: 
            expenses = cleaning_cost + room_cost + room_cost*0.2
        
        if deposite == 'on' and row['predictions'] == 1:
            revenue = expenses*0.8
        else:
            revenue = 0

    return revenue - expenses
    
    

In [33]:
df = data_test.copy()
df['profit'] = prices(df).apply(profit, deposite='off', axis=1)
print('Общая прибыль гостиницы без внедрения депозитов за 2017 год:', df['profit'].sum(), 'рублей.')
print('Из-за отмены бронирования гостиница потеряла за 2017 год:', abs(df.query('is_canceled == 1')['profit'].sum()), 'рублей.')

Общая прибыль гостиницы без внедрения депозитов за 2017 год: 32683030.0 рублей.
Из-за отмены бронирования гостиница потеряла за 2017 год: 10571770.0 рублей.


### Вывод по шагу 4.

За 2017 год прибыль составляет 32 млн. рублей, 10 миллионов гостиница потеряла из-за отмен.

## Разработка модели ML <a id="4"></a>

### Подготовка данных.

In [34]:
target_train = data_train['is_canceled']
features_train = data_train.drop('is_canceled', axis=1)
target_test = data_test['is_canceled']
features_test = data_test.drop('is_canceled', axis=1)

In [35]:
numeric = ['lead_time', 'arrival_date_year', 'arrival_date_week_number', 'arrival_date_day_of_month',  'adults', 'children', 'babies', 'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled', 'booking_changes', 'days_in_waiting_list', 'required_car_parking_spaces', 'total_of_special_requests', 'total_nights']
cat = ['arrival_date_month', 'meal', 'country', 'distribution_channel', 'reserved_room_type', 'customer_type']

In [36]:
encoder = OrdinalEncoder() 
scaler = StandardScaler() 
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

tree_features_train = encoder.fit_transform(features_train, target_train)
tree_features_test = encoder.fit_transform(features_test, target_test)

features_train[numeric] = scaler.fit_transform(features_train[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])
lg_features_train = ohe.fit_transform(features_train, target_train)
lg_features_test = ohe.transform(features_test)



print(tree_features_train.shape, tree_features_test.shape)
print(target_train.shape, target_test.shape)

print(lg_features_train.shape, lg_features_test.shape)
print(target_train.shape, target_test.shape)

(43852, 21) (24695, 21)
(43852,) (24695,)
(43852, 862) (24695, 862)
(43852,) (24695,)


In [37]:
target_train.value_counts()

0    32704
1    11148
Name: is_canceled, dtype: int64

Попробуем обучить три модели - дерево решений, случайный лес, логистическую регрессию без и с исправлением дисбаланса.

Для получения максимальной прибыли нам необходимо минимизировать количество ложноотрицательных ответов модели, поэтому подбирать лучшую по качеству будем по метрике recall. Чем меньше ложноотрицательных срабатываний, тем выше recall модели.

### Дерево решений.

Настроим гиперпараметры.

In [38]:
'''
best_depth = 0
best_recall = 0

for depth in range(1,101):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(tree_features_train, target_train)
    predicted = model.predict(tree_features_test)
    recall = recall_score(target_test, predicted)
    if recall > best_recall:
        best_depth = depth
        best_recall = recall

print('Лучшая глубина дерева:', best_depth)
print('Лучшее значение recall:', best_recall)  
'''

"\nbest_depth = 0\nbest_recall = 0\n\nfor depth in range(1,101):\n    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)\n    model.fit(tree_features_train, target_train)\n    predicted = model.predict(tree_features_test)\n    recall = recall_score(target_test, predicted)\n    if recall > best_recall:\n        best_depth = depth\n        best_recall = recall\n\nprint('Лучшая глубина дерева:', best_depth)\nprint('Лучшее значение recall:', best_recall)  \n"

In [39]:
'''
best_ss = 0
best_recall = 0

for ss in range(2,101):
    model = DecisionTreeClassifier(random_state=12345, max_depth=5, min_samples_split=ss)
    model.fit(tree_features_train, target_train)
    predicted = model.predict(tree_features_test)
    recall = recall_score(target_test, predicted)
    if recall > best_recall:
        best_ss = ss
        best_recall = recall

print('Лучшee значение параметра min_samples_split:', best_ss)
print('Лучшее значение recall:', best_recall) 
'''

"\nbest_ss = 0\nbest_recall = 0\n\nfor ss in range(2,101):\n    model = DecisionTreeClassifier(random_state=12345, max_depth=5, min_samples_split=ss)\n    model.fit(tree_features_train, target_train)\n    predicted = model.predict(tree_features_test)\n    recall = recall_score(target_test, predicted)\n    if recall > best_recall:\n        best_ss = ss\n        best_recall = recall\n\nprint('Лучшee значение параметра min_samples_split:', best_ss)\nprint('Лучшее значение recall:', best_recall) \n"

In [40]:
'''
best_ll = 0
best_recall = 0

for ll in range(1,101):
    model = DecisionTreeClassifier(random_state=12345, max_depth=5, min_samples_split=2,min_samples_leaf=ll)
    model.fit(tree_features_train, target_train)
    predicted = model.predict(tree_features_test)
    recall = recall_score(target_test, predicted)
    if recall > best_recall:
        best_ll = ll
        best_recall = recall

print('Лучшee значение параметра min_samples_leaf:', best_ll)
print('Лучшее значение recall:', best_recall) 
'''

"\nbest_ll = 0\nbest_recall = 0\n\nfor ll in range(1,101):\n    model = DecisionTreeClassifier(random_state=12345, max_depth=5, min_samples_split=2,min_samples_leaf=ll)\n    model.fit(tree_features_train, target_train)\n    predicted = model.predict(tree_features_test)\n    recall = recall_score(target_test, predicted)\n    if recall > best_recall:\n        best_ll = ll\n        best_recall = recall\n\nprint('Лучшee значение параметра min_samples_leaf:', best_ll)\nprint('Лучшее значение recall:', best_recall) \n"

In [41]:
'''
tree_model = DecisionTreeClassifier(random_state=12345, max_depth=5, min_samples_split=2,min_samples_leaf=1)
tree_model.fit(tree_features_train, target_train)
predicted = tree_model.predict(tree_features_test)
scores = cross_val_score(tree_model, tree_features_test, target_test, cv=5, scoring='recall')
final_score = sum(scores) / len(scores)
final_score
'''

"\ntree_model = DecisionTreeClassifier(random_state=12345, max_depth=5, min_samples_split=2,min_samples_leaf=1)\ntree_model.fit(tree_features_train, target_train)\npredicted = tree_model.predict(tree_features_test)\nscores = cross_val_score(tree_model, tree_features_test, target_test, cv=5, scoring='recall')\nfinal_score = sum(scores) / len(scores)\nfinal_score\n"

In [42]:
tree_model1 = DecisionTreeClassifier(random_state=12345, max_depth=5, min_samples_split=2,min_samples_leaf=1, class_weight='balanced')
tree_model1.fit(tree_features_train, target_train)
predicted = tree_model1.predict(tree_features_test)
scores = cross_val_score(tree_model1, tree_features_test, target_test, cv=5, scoring='recall')
final_score = sum(scores) / len(scores)
final_score

0.6633269107257547

### Случайный лес.

In [43]:
'''
best_estim = 0
best_recall = 0

for estim in range(1,101):
    model = RandomForestClassifier(random_state=12345, n_estimators=estim)
    model.fit(tree_features_train, target_train)
    predicted = model.predict(tree_features_test)
    recall = recall_score(target_test, predicted)
    
    if recall > best_recall:
        best_estim = estim
        best_recall = recall

print('Лучшее количество деревьев:', best_estim)
print('Лучшее значение recall:', best_recall)  
'''

"\nbest_estim = 0\nbest_recall = 0\n\nfor estim in range(1,101):\n    model = RandomForestClassifier(random_state=12345, n_estimators=estim)\n    model.fit(tree_features_train, target_train)\n    predicted = model.predict(tree_features_test)\n    recall = recall_score(target_test, predicted)\n    \n    if recall > best_recall:\n        best_estim = estim\n        best_recall = recall\n\nprint('Лучшее количество деревьев:', best_estim)\nprint('Лучшее значение recall:', best_recall)  \n"

In [44]:
'''
ff = ['sqrt', 'log2', 'auto']

for f in ff:
    model = RandomForestClassifier(random_state=12345, n_estimators=1, max_features=f)
    model.fit(tree_features_train, target_train)
    predicted = model.predict(tree_features_test)
    recall = recall_score(target_test, predicted)
    
    print(f)
    print(recall)  
'''

"\nff = ['sqrt', 'log2', 'auto']\n\nfor f in ff:\n    model = RandomForestClassifier(random_state=12345, n_estimators=1, max_features=f)\n    model.fit(tree_features_train, target_train)\n    predicted = model.predict(tree_features_test)\n    recall = recall_score(target_test, predicted)\n    \n    print(f)\n    print(recall)  \n"

In [45]:
'''
cc = ['gini', 'entropy']
for c in cc:
    model = RandomForestClassifier(random_state=12345, n_estimators=1, criterion=c)
    model.fit(tree_features_train, target_train)
    predicted = model.predict(tree_features_test)
    recall = recall_score(target_test, predicted)
    
    print(c)
    print(recall) 
'''

"\ncc = ['gini', 'entropy']\nfor c in cc:\n    model = RandomForestClassifier(random_state=12345, n_estimators=1, criterion=c)\n    model.fit(tree_features_train, target_train)\n    predicted = model.predict(tree_features_test)\n    recall = recall_score(target_test, predicted)\n    \n    print(c)\n    print(recall) \n"

In [46]:
'''
forest_model = RandomForestClassifier(random_state=12345,n_estimators=1, criterion='entropy')
forest_model.fit(tree_features_train, target_train)  
predicted = forest_model.predict(tree_features_test)
scores = cross_val_score(forest_model, tree_features_test, target_test,  cv=5, scoring='recall')
final_score = sum(scores) / len(scores)
final_score
'''

"\nforest_model = RandomForestClassifier(random_state=12345,n_estimators=1, criterion='entropy')\nforest_model.fit(tree_features_train, target_train)  \npredicted = forest_model.predict(tree_features_test)\nscores = cross_val_score(forest_model, tree_features_test, target_test,  cv=5, scoring='recall')\nfinal_score = sum(scores) / len(scores)\nfinal_score\n"

In [47]:
'''
forest_model1 = RandomForestClassifier(random_state=12345,n_estimators=1, criterion='entropy', class_weight='balanced')
forest_model1.fit(tree_features_train, target_train)  
predicted = forest_model1.predict(tree_features_test)
scores = cross_val_score(forest_model1, tree_features_test, target_test,  cv=5, scoring='recall')
final_score = sum(scores) / len(scores)
final_score
'''

"\nforest_model1 = RandomForestClassifier(random_state=12345,n_estimators=1, criterion='entropy', class_weight='balanced')\nforest_model1.fit(tree_features_train, target_train)  \npredicted = forest_model1.predict(tree_features_test)\nscores = cross_val_score(forest_model1, tree_features_test, target_test,  cv=5, scoring='recall')\nfinal_score = sum(scores) / len(scores)\nfinal_score\n"

### Логистическая регрессия

In [48]:
'''
pp = ['lbfgs', 'liblinear', 'newton-cg', 'sag', 'saga']
for p in pp:
    model = LogisticRegression(random_state=12345, solver=p)
    model.fit(lg_features_train, target_train)
    predicted = model.predict(lg_features_test)
    recall = recall_score(target_test, predicted)
    
    print(p)
    print(recall)  
    
'''

"\npp = ['lbfgs', 'liblinear', 'newton-cg', 'sag', 'saga']\nfor p in pp:\n    model = LogisticRegression(random_state=12345, solver=p)\n    model.fit(lg_features_train, target_train)\n    predicted = model.predict(lg_features_test)\n    recall = recall_score(target_test, predicted)\n    \n    print(p)\n    print(recall)  \n    \n"

lbfgs
0.35863840719332046
liblinear
0.38098908156711625
newton-cg
0.20192678227360308
sag
0.20655105973025048
saga
0.23596660244059087

In [49]:
'''
max_iter = 0
best_recall = 0

for i in range(1,101):
    model = LogisticRegression(random_state=12345, solver='liblinear', max_iter=i)
    model.fit(lg_features_train, target_train)
    predicted = model.predict(lg_features_test)
    recall = recall_score(target_test, predicted)
    
    if recall > best_recall:
        best_iter = i
        best_recall = recall

print('best_iter:', best_iter)
print('Лучшее значение recall:', best_recall)  
'''

"\nmax_iter = 0\nbest_recall = 0\n\nfor i in range(1,101):\n    model = LogisticRegression(random_state=12345, solver='liblinear', max_iter=i)\n    model.fit(lg_features_train, target_train)\n    predicted = model.predict(lg_features_test)\n    recall = recall_score(target_test, predicted)\n    \n    if recall > best_recall:\n        best_iter = i\n        best_recall = recall\n\nprint('best_iter:', best_iter)\nprint('Лучшее значение recall:', best_recall)  \n"

best_iter: 10
Лучшее значение recall: 0.38098908156711625

In [50]:
'''
log_model =  LogisticRegression(random_state=12345, solver='liblinear', max_iter=10)
log_model.fit(lg_features_train, target_train)
predicted_valid = log_model.predict(lg_features_test)
scores = cross_val_score(log_model, tree_features_test, target_test,  cv=5, scoring='recall')
final_score = sum(scores) / len(scores)
final_score
'''

"\nlog_model =  LogisticRegression(random_state=12345, solver='liblinear', max_iter=10)\nlog_model.fit(lg_features_train, target_train)\npredicted_valid = log_model.predict(lg_features_test)\nscores = cross_val_score(log_model, tree_features_test, target_test,  cv=5, scoring='recall')\nfinal_score = sum(scores) / len(scores)\nfinal_score\n"

In [51]:
log_model1 =  LogisticRegression(random_state=12345, solver='liblinear', max_iter=10, class_weight='balanced')
log_model1.fit(lg_features_train, target_train)
predicted_test = log_model1.predict(lg_features_test)
scores = cross_val_score(log_model1, tree_features_test, target_test,  cv=5, scoring='recall')
final_score = sum(scores) / len(scores)
final_score

0.6296724470134875

Лучшая средняя оценка качества 0.6633269107257547 у дерева решений с балансом классов.

### Проверка на тестовой выборке.

In [52]:
predicted = tree_model1.predict(tree_features_test)

In [53]:
print('Accuracy:', accuracy_score(target_test, predicted))
print('Precision:', precision_score(target_test, predicted))
print('Recall:', recall_score(target_test, predicted))
print('F1:', f1_score(target_test, predicted))

Accuracy: 0.6719173921846527
Precision: 0.34536585365853656
Recall: 0.045472061657032756
F1: 0.08036322360953461


In [54]:
predicted_lg = log_model1.predict(lg_features_test)

In [55]:
print('Accuracy:', accuracy_score(target_test, predicted_lg))
print('Precision:', precision_score(target_test, predicted_lg))
print('Recall:', recall_score(target_test, predicted_lg))
print('F1:', f1_score(target_test, predicted_lg))

Accuracy: 0.6974286292771816
Precision: 0.5149546106067845
Recall: 0.692228644829801
F1: 0.5905753424657535


Целевая метрика recall, но сравнение по другим метрикам делает логистическую регрессию более презентабельной. Проверим roc-auc и применим функцию расчета прибыли к данным, полученным из обеих моделей.

In [56]:
'''
plt.figure(figsize=[12,9])

plt.plot([0, 1], [0, 1], linestyle='--', label='RandomModel')



probabilities_test = tree_model1.predict_proba(tree_features_test)
probabilities_one_valid = probabilities_test[:, 1]
fpr, tpr, thresholds = roc_curve(target_test, probabilities_one_valid)
auc_roc = roc_auc_score(target_test, probabilities_one_valid)
print('AUC-ROC DecisionTreeClassifier',auc_roc)
plt.plot(fpr, tpr, label='DecisionTreeClassifier')



probabilities_test_lg = log_model1.predict_proba(lg_features_test)
probabilities_one_valid = probabilities_test_lg[:, 1]
fpr, tpr, thresholds = roc_curve(target_test, probabilities_one_valid)
auc_roc = roc_auc_score(target_test, probabilities_one_valid)
print('AUC-ROC LogisticRegression',auc_roc)
plt.plot(fpr, tpr, label='LogisticRegression')


plt.xlim([0,1])
plt.ylim([0,1])

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

plt.legend(loc='lower right', fontsize='x-large')

plt.title("ROC-кривая")
plt.show()
'''

'\nplt.figure(figsize=[12,9])\n\nplt.plot([0, 1], [0, 1], linestyle=\'--\', label=\'RandomModel\')\n\n\n\nprobabilities_test = tree_model1.predict_proba(tree_features_test)\nprobabilities_one_valid = probabilities_test[:, 1]\nfpr, tpr, thresholds = roc_curve(target_test, probabilities_one_valid)\nauc_roc = roc_auc_score(target_test, probabilities_one_valid)\nprint(\'AUC-ROC DecisionTreeClassifier\',auc_roc)\nplt.plot(fpr, tpr, label=\'DecisionTreeClassifier\')\n\n\n\nprobabilities_test_lg = log_model1.predict_proba(lg_features_test)\nprobabilities_one_valid = probabilities_test_lg[:, 1]\nfpr, tpr, thresholds = roc_curve(target_test, probabilities_one_valid)\nauc_roc = roc_auc_score(target_test, probabilities_one_valid)\nprint(\'AUC-ROC LogisticRegression\',auc_roc)\nplt.plot(fpr, tpr, label=\'LogisticRegression\')\n\n\nplt.xlim([0,1])\nplt.ylim([0,1])\n\nplt.xlabel("False Positive Rate")\nplt.ylabel("True Positive Rate")\n\nplt.legend(loc=\'lower right\', fontsize=\'x-large\')\n\nplt.t

Получили неплохое значение метрики roc-auc у логистической регрессии.

Ранее мы получили следующие данные:

Общая прибыль гостиницы без внедрения депозитов за 2017 год: 32 683030 рублей.

Из-за отмены бронирования гостиница потеряла за 2017 год: 10 571770 рублей.

In [57]:
new_data = data_test.copy()
new_data['predictions'] = predicted

In [58]:

new_data['profit'] = prices(new_data).apply(profit, deposite='on', axis=1)

print('Общая прибыль гостиницы с внедрением депозитов за 2017 год:', new_data['profit'].sum(), 'рублей.')
print('Потери гостиницы с учетом ошибок модели составят:', abs(new_data.query('is_canceled == 1')['profit'].sum()), 'рублей.')
print('Модель дерева решений поможет сохранить:', abs(df.query('is_canceled == 1')['profit'].sum()) - abs(new_data.query('is_canceled == 1')['profit'].sum()), 'рублей.')


Общая прибыль гостиницы с внедрением депозитов за 2017 год: 33065182.0 рублей.
Потери гостиницы с учетом ошибок модели составят: 10189618.0 рублей.
Модель дерева решений поможет сохранить: 382152.0 рублей.


In [59]:
new_data_lg = data_test.copy()
new_data_lg['predictions'] = predicted_lg

In [60]:

new_data_lg['profit'] = prices(new_data_lg).apply(profit, deposite='on', axis=1)
print('Общая прибыль гостиницы с внедрением депозитов за 2017 год:', new_data_lg['profit'].sum(), 'рублей.')
print('Потери гостиницы с учетом ошибок модели составят:', abs(new_data_lg.query('is_canceled == 1')['profit'].sum()), 'рублей.')
print('Модель логистической регрессии поможет сохранить:', abs(df.query('is_canceled == 1')['profit'].sum()) - abs(new_data_lg.query('is_canceled == 1')['profit'].sum()), 'рублей.')


Общая прибыль гостиницы с внедрением депозитов за 2017 год: 38572574.0 рублей.
Потери гостиницы с учетом ошибок модели составят: 4682226.0 рублей.
Модель логистической регрессии поможет сохранить: 5889544.0 рублей.


На разработку системы прогнозирования заложен бюджет — 400 000. Окупается только модель логистической регрессии.

### Вывод по шагу 5.
Было обучено три модели: решающее дерево, случайный лес и логистическая регрессия. После устранения дисбаланса в классах лучший результат по средней оценке качества получили у модели случайного леса - 0.8616.

Модель была применена на тестовой  выборке, значение метрики roc-auc =  0.807.

Функция подсчета прибыли применена на пресказанных результатах. Прибыль бизнеса с внедрением модели увеличится на почти 6 миллионов.

## Портрет «ненадёжного» клиента <a id="5"></a>

In [61]:
data_ohe = pd.get_dummies(data_train, drop_first=True)

In [62]:
target = data_ohe['is_canceled']
features = data_ohe.drop('is_canceled', axis=1)

features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, 
                                                                              random_state=12345, stratify=target)


scaler = StandardScaler(with_mean=False)
features_train[numeric] = scaler.fit_transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])



print(features_train.shape, features_valid.shape)
print(target_train.shape, target_valid.shape)


(32889, 195) (10963, 195)
(32889,) (10963,)


In [63]:
model = RandomForestClassifier(random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)

Рассмотрим долю вклада каждого признака в итоговое предстказание.

In [64]:
#feat_import = pd.DataFrame(data = {'feature': features_train.columns, 'percent': model.feature_importances_})
#feat_import.sort_values('percent', ascending=False).reset_index(drop=True)

In [65]:
#data_test.query('is_canceled==1').describe().T

In [66]:
#data_test.query('is_canceled==1')['total_nights'].value_counts()

Наиболее важные признаки ненадежного клиента:

- чем больше дней до заезда остается, тем вероятнее отмена бронирования
- гости из Португалии отменяют бронирование чаще остальных
- ненадежным является клиент с большим количеством специальных отметок
- если клиент уже отменял бронирование раньше, вероятнее всего он отменит его и в этот раз
- в среднем ненадежный клиент бронирует номер на 3-4 дня


## Общий вывод <a id="6"></a>

В ходе исследования данные были подготовлены для обучения модели: удалены столбцы, не имеющие значимость, изменены типы данных, заменены значения в столбцах.

Написана функция подсчета прибыли с учетом внедрения депозита и без.

Обучены модели, на основе показателя лучшей средней оценки выбрана модель случайного леса для предсказания отмены бронирования. Метрика roc-auc равна 0.807.

Использование модели в бизнесе может помочь увеличить прибыль гостиницы в два раза. Выделенный на разработку бюджет окупается.

Составлен портрет "ненадежного клиента".